In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
import pandas as pd
from random import sample
import random
import string
import json
import gc  # Python's garbage collector

In [3]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from datasets import Dataset
from huggingface_hub import create_repo

In [5]:
from huggingface_hub import notebook_login

notebook_login()

## Symbol Generator

In [6]:
json_path = '/content/drive/MyDrive/Phi-2/data/evaluation_unique_symbols.json'

def generate_random_word(length=5):
    vowels = "aeiou"
    consonants = "".join(set(string.ascii_lowercase) - set(vowels))

    # Create a word-like string by alternating between consonants and vowels
    word = ""
    for i in range(length):
        if i % 2 == 0:
            word += random.choice(consonants)
        else:
            word += random.choice(vowels)
    return word.capitalize()


def generate_random_integer():
    return str(random.randint(10000, 99999))

def generate_random_uppercase():
    return ''.join(random.choices(string.ascii_uppercase, k=4))

def read_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def write_json(file_path, data):
    with open(file_path, 'w') as file:
        json.dump(data, file)

def generate_unique_symbol(json_path=json_path):
    data = read_json(json_path)
    unique_symbols = data.get("unique_symbols", [])

    # Generate a unique symbol
    symbol_generated = False
    while not symbol_generated:
        category = random.choice([generate_random_integer, generate_random_uppercase, generate_random_word])
        symbol = category()
        if symbol not in unique_symbols:
            # unique_symbols.append(symbol)
            symbol_generated = True

    # Update the JSON file
    #data["unique_symbols"] = unique_symbols
    #write_json(json_path, data)
    return symbol

In [39]:
generate_unique_symbol()

'XAKH'

## SUBJ

- With Instruction, No Relevant Label
- No Instruction, No Relevant Label
- Flipped Data

In [35]:
subj_data = load_dataset("tasksource/subjectivity")

In [36]:
subj_data

DatasetDict({
    train: Dataset({
        features: ['Sentence', 'Label', 'Solved conflict'],
        num_rows: 731
    })
    validation: Dataset({
        features: ['Sentence', 'Label', 'Solved conflict'],
        num_rows: 99
    })
    test: Dataset({
        features: ['Sentence', 'Label', 'Solved conflict'],
        num_rows: 219
    })
})

In [37]:
subj = [item for item in subj_data['train'] if item['Label'] == "SUBJ"]
no_subj = [item for item in subj_data['train'] if item['Label'] == "OBJ"]
print(f"subj: {len(subj)}")
print(f"no_subj: {len(no_subj)}")

subj: 244
no_subj: 487


In [55]:
# Select the first 500 examples from the training set
subset = subj_data['train'].select(range(500))

# Shuffle the data
subset = subset.shuffle(seed=42)

subj = [item for item in subset if item['Label'] == "SUBJ"]
no_subj = [item for item in subset if item['Label'] == "OBJ"]
print(f"subj: {len(subj)}")
print(f"no_subj: {len(no_subj)}")

subj: 167
no_subj: 333


### Build the subset


In [58]:
def build_examples_dataset(data, num_positives=4, num_negatives=4):
    examples_dataset = []
    count = 0
    for idx, target in enumerate(data):
        # Filter out the target text from the lists
        positive_examples = [item for item in data if item['Label'] == "SUBJ" and item['Sentence'] != target['Sentence']] # subjective
        negative_examples = [item for item in data if item['Label'] == "OBJ" and item['Sentence'] != target['Sentence']] # objective

        # random select positive and negative
        selected_positives = random.sample(positive_examples, num_positives)
        selected_negatives = random.sample(negative_examples, num_negatives)

        # Assuming 'text' is your question and 'label' gives the answer (hate/no hate)
        example = {
            'positives': [ex['Sentence'] for ex in selected_positives],
            'negatives': [ex['Sentence'] for ex in selected_negatives],
            'target_question': target['Sentence'],
            'target_answer': "positives" if target['Label'] == "SUBJ" else "negatives"
        }

        examples_dataset.append(example)
        print(count)
        count += 1

    return examples_dataset

In [59]:
processed_dataset = build_examples_dataset(subset)
processed_dataset[0]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

{'positives': ['The loss of public credit, the complete ruin of it, would be the least of the consequences.\n',
  'Regardless of whether this tentative agreement is approved or not, deep issues with the rail system will remain.\n',
  'If you’re looking for a bellwether for the world economy, you could do worse than consider what’s happening at the very highest end of London’s property market.\n',
  'As this logistics system becomes more interdependent and fragile, the potential leverage of these workers increases to a dramatic level.\n'],
 'negatives': ['Already of those above ten years of age gainfully employed in the whole country, male and female, about one in ten is directly employed in government service.\n',
  'It is a measure of a person’s immune system performance.\n',
  'The city, of course, is an important factor.\n',
  'The structure of government is by strata, beginning with innumerable small local units, such as boroughs, townships, school districts, improvement districts,

In [60]:
# Save the dataset to a JSON file
file_path = '/content/drive/MyDrive/Phi-2/data/eval/subjective/subjective_dataset.json'
with open(file_path, 'w') as f:
    json.dump(processed_dataset, f)

### Build the With Instruction and With Relavent Label

In [61]:
def build_dataframe_with_instruct_label(dict_list, instruction, instruction_tag, answer_tag, answer_pos, answer_neg):
    # Transform each entry into the desired format for "inputs" and "target"
    transformed_data = []
    input_text = ""
    for item in dict_list:
        combined = item['positives'] + item['negatives']
        random.shuffle(combined)  # Randomly shuffle the positive and negative examples

        for ex in combined:
          input_text += f"{instruction_tag}: {instruction}\n"
          input_text += f"{ex}\n"
          answer = f'{answer_tag}: {answer_pos}' if ex in item['positives'] else f'{answer_tag}: {answer_neg}'
          input_text += f'{answer}\n'

        # The target answer remains the same
        input_text += f"{instruction_tag}: {instruction}\n"
        input_text += f"{item['target_question']}\n"
        input_text += f'{answer_tag}: \n'
        target = answer_pos if item['target_answer'] == "positives" else answer_neg

        transformed_data.append({'inputs': input_text, 'target': target})
        input_text = ""

    # Create a DataFrame
    df = pd.DataFrame(transformed_data)
    return df

In [62]:
df_with_instruct_label = build_dataframe_with_instruct_label(
    dict_list=processed_dataset,
    instruction="Is the following sentence subjective or objective?",
    instruction_tag="Question",
    answer_tag="Answer",
    answer_pos="Subjective",
    answer_neg="Objective"
)

In [63]:
# Save the DataFrame as a pickle file
pickle_path = '/content/drive/MyDrive/Phi-2/data/eval/subjective/subjective_with_instruct_label.pkl'
df_with_instruct_label.to_pickle(pickle_path)

In [64]:
# Assuming df_with_instruct_label is your pandas DataFrame
dataset = Dataset.from_pandas(df_with_instruct_label)

In [65]:
repo_name = "subjective-with-instruction-with-label"  # Choose a unique name
create_repo(repo_name, repo_type="dataset", private=False)  # Set `private=True` for a private dataset

# Push the train dataset
dataset.push_to_hub("AlanYky/subjective-with-instruction-with-label")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/AlanYky/subjective-with-instruction-with-label/commit/cf31c2b60bb85553b1d56f18204de63c40ee056c', commit_message='Upload dataset', commit_description='', oid='cf31c2b60bb85553b1d56f18204de63c40ee056c', pr_url=None, pr_revision=None, pr_num=None)

### Build the With Instruction and No Relavent Label

In [66]:
def build_dataframe_with_instruct_no_label(dict_list, instruction, instruction_tag, answer_tag):
    # Transform each entry into the desired format for "inputs" and "target"
    transformed_data = []
    input_text = ""
    for item in dict_list:
        answer_pos = generate_unique_symbol()
        answer_neg = generate_unique_symbol()

        combined = item['positives'] + item['negatives']
        random.shuffle(combined)  # Randomly shuffle the positive and negative examples

        for ex in combined:
          input_text += f"{instruction_tag}: {instruction}\n"
          input_text += f"{ex}\n"
          answer = f'{answer_tag}: {answer_pos}' if ex in item['positives'] else f'{answer_tag}: {answer_neg}'
          input_text += f'{answer}\n'

        # The target answer remains the same
        input_text += f"{instruction_tag}: {instruction}\n"
        input_text += f"{item['target_question']}\n"
        input_text += f'{answer_tag}: \n'
        target = answer_pos if item['target_answer'] == "positives" else answer_neg

        transformed_data.append({'inputs': input_text, 'target': target})
        input_text = ""

    # Create a DataFrame
    df = pd.DataFrame(transformed_data)
    return df

In [67]:
df_with_instruct_no_label = build_dataframe_with_instruct_no_label(
    dict_list=processed_dataset,
    instruction="Is the following sentence subjective or objective?",
    instruction_tag="Question",
    answer_tag="Answer",
)


In [68]:
# Save the DataFrame as a pickle file
pickle_path = '/content/drive/MyDrive/Phi-2/data/eval/subjective/subjective_with_instruct_symbol.pkl'
df_with_instruct_no_label.to_pickle(pickle_path)

In [69]:
# Assuming df_with_instruct_label is your pandas DataFrame
dataset = Dataset.from_pandas(df_with_instruct_no_label)

In [70]:
repo_name = "subjective-with-instruction-with-symbol"  # Choose a unique name
create_repo(repo_name, repo_type="dataset", private=False)  # Set `private=True` for a private dataset

# Push the train dataset
dataset.push_to_hub("AlanYky/subjective-with-instruction-with-symbol")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/AlanYky/subjective-with-instruction-with-symbol/commit/66838cfec638334f48418f837bacbfad2c9e5896', commit_message='Upload dataset', commit_description='', oid='66838cfec638334f48418f837bacbfad2c9e5896', pr_url=None, pr_revision=None, pr_num=None)

### Build the No Instruction and No Relavent Label

In [71]:
def build_dataframe_no_instruct_no_label(dict_list, instruction_tag, answer_tag):
    # Transform each entry into the desired format for "inputs" and "target"
    transformed_data = []
    input_text = ""
    for item in dict_list:
        answer_pos = generate_unique_symbol()
        answer_neg = generate_unique_symbol()

        combined = item['positives'] + item['negatives']
        random.shuffle(combined)  # Randomly shuffle the positive and negative examples

        for ex in combined:
          input_text += f"{instruction_tag}: {ex}\n"
          answer = f'{answer_tag}: {answer_pos}' if ex in item['positives'] else f'{answer_tag}: {answer_neg}'
          input_text += f'{answer}\n'

        # The target answer remains the same
        input_text += f"{instruction_tag}: {item['target_question']}\n"
        input_text += f'{answer_tag}: \n'
        target = answer_pos if item['target_answer'] == "positives" else answer_neg

        transformed_data.append({'inputs': input_text, 'target': target})
        input_text = ""

    # Create a DataFrame
    df = pd.DataFrame(transformed_data)
    return df

In [72]:
df_with_instruct_no_instruct_no_label = build_dataframe_no_instruct_no_label(
    dict_list=processed_dataset,
    instruction_tag="Question",
    answer_tag="Answer",
)

In [73]:
# Save the DataFrame as a pickle file
pickle_path = '/content/drive/MyDrive/Phi-2/data/eval/subjective/subjective_no_instruct_symbol.pkl'
df_with_instruct_no_instruct_no_label.to_pickle(pickle_path)

In [74]:
# Assuming df_with_instruct_label is your pandas DataFrame
dataset = Dataset.from_pandas(df_with_instruct_no_instruct_no_label)

In [75]:
repo_name = "subjective-no-instruction-with-symbol"  # Choose a unique name
create_repo(repo_name, repo_type="dataset", private=False)  # Set `private=True` for a private dataset

# Push the train dataset
dataset.push_to_hub("AlanYky/subjective-no-instruction-with-symbol")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/AlanYky/subjective-no-instruction-with-symbol/commit/692a9964c1b107c6072d162574fb73f5693d2594', commit_message='Upload dataset', commit_description='', oid='692a9964c1b107c6072d162574fb73f5693d2594', pr_url=None, pr_revision=None, pr_num=None)

### Flipped Data

In [76]:
def build_dataframe_flipped(dict_list, instruction_tag, answer_tag, answer_pos, answer_neg):
    # Transform each entry into the desired format for "inputs" and "target"
    transformed_data = []
    input_text = ""
    for item in dict_list:
        combined = item['positives'] + item['negatives']
        random.shuffle(combined)  # Randomly shuffle the positive and negative examples

        for ex in combined:
          input_text += f"{instruction_tag}: {ex}\n"
          answer = f'{answer_tag}: {answer_neg}' if ex in item['positives'] else f'{answer_tag}: {answer_pos}' # Flipped Here
          input_text += f'{answer}\n'

        # The target answer remains the same
        input_text += f"{instruction_tag}: {item['target_question']}\n"
        input_text += f'{answer_tag}: \n'
        target = answer_neg if item['target_answer'] == "positives" else answer_pos

        transformed_data.append({'inputs': input_text, 'target': target})
        input_text = ""

    # Create a DataFrame
    df = pd.DataFrame(transformed_data)
    return df

In [77]:
df_with_flipped = build_dataframe_flipped(
    dict_list=processed_dataset,
    instruction_tag="Input",
    answer_tag="Symbol",
    answer_pos="Subjective",
    answer_neg="Objective"
)

In [78]:
# Save the DataFrame as a pickle file
pickle_path = '/content/drive/MyDrive/Phi-2/data/eval/subjective/subjective_flipped.pkl'
df_with_flipped.to_pickle(pickle_path)

In [79]:
# Assuming df_with_instruct_label is your pandas DataFrame
dataset = Dataset.from_pandas(df_with_flipped)

In [80]:
repo_name = "subjective-flipped"  # Choose a unique name
create_repo(repo_name, repo_type="dataset", private=False)  # Set `private=True` for a private dataset

# Push the train dataset
dataset.push_to_hub("AlanYky/subjective-flipped")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/AlanYky/subjective-flipped/commit/a967330b2bc6db09db73a261a4709e4780d951d2', commit_message='Upload dataset', commit_description='', oid='a967330b2bc6db09db73a261a4709e4780d951d2', pr_url=None, pr_revision=None, pr_num=None)

## TEH

- With Instruction, No Relevant Label
- No Instruction, No Relevant Label
- Flipped Data

In [17]:
hate_data = load_dataset("tweet_eval", "hate")

In [28]:
hate_data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 9000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2970
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
})

In [31]:
hate = [item for item in hate_data['train'] if item['label'] == 1]
no_hate = [item for item in hate_data['train'] if item['label'] == 0]
print(f"hate: {len(hate)}")
print(f"no_hate: {len(no_hate)}")

hate: 3783
no_hate: 5217


In [32]:
# Select the first 500 examples from the training set
subset = hate_data['train'].select(range(2000))

# Shuffle the data
subset = subset.shuffle(seed=42)

hate = [item for item in subset if item['label'] == 1]
no_hate = [item for item in subset if item['label'] == 0]
print(f"hate: {len(hate)}")
print(f"no_hate: {len(no_hate)}")

hate: 827
no_hate: 1173


In [11]:
# Attempt to free up memory
del hate_data  # Delete the original large dataset variable
gc.collect()  # Explicitly call garbage collector

5

### Build the subset

In [35]:
def build_examples_dataset(data, num_positives=4, num_negatives=4):
    examples_dataset = []
    count = 0
    for idx, target in enumerate(data):
        # Filter out the target text from the lists
        positive_examples = [item for item in data if item['label'] == 1 and item['text'] != target['text']] # hate
        negative_examples = [item for item in data if item['label'] == 0 and item['text'] != target['text']] # non hate

        # random select positive and negative
        selected_positives = random.sample(positive_examples, num_positives)
        selected_negatives = random.sample(negative_examples, num_negatives)

        # Assuming 'text' is your question and 'label' gives the answer (hate/no hate)
        example = {
            'positives': [ex['text'] for ex in selected_positives],
            'negatives': [ex['text'] for ex in selected_negatives],
            'target_question': target['text'],
            'target_answer': "positives" if target['label'] == 1 else "negatives"
        }

        examples_dataset.append(example)
        print(count)
        count += 1

    return examples_dataset

In [36]:
processed_dataset = build_examples_dataset(subset)
processed_dataset[0]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

{'positives': ['I just wanna know why the girl dont like me 😭😂 What I ever do to you hoe? Other than give you rides and be nice to you 😂🤦🏻\u200d♀️ Hating ass hoe',
  'There are good men out there women stop accepting ass holes that appear to be right.It is a excuse to turn lesbian cuz of a no good man.',
  '@user @user @user @user @user #BuildThatWall to end #Corruption and then deport, deport, and deport.',
  'Was getting elbowed by a triple chinned cunt with nails like a woman ont train Let a smelly fart out before moving…'],
 'negatives': ['@user @user @user I always use Japan or Poland as an example of nationalism done right. Very little immigration to those countries and zero terrorism taking place there.',
  "I've never liked @user and he didn't endear himself with the Sean Spicer bromance and Stupid Weistein jokes. Bit of a cunt",
  "If they don't trust you enough to let you cheat in peace is the relationship really real?",
  'Welcome to Germany, and you get three weeks paid vac

In [37]:
# Save the dataset to a JSON file
file_path = '/content/drive/MyDrive/Phi-2/data/eval/hate/hate_dataset.json'
with open(file_path, 'w') as f:
    json.dump(processed_dataset, f)

### Build the With Instruction and With Relavent Label

In [38]:
def build_dataframe_with_instruct_label(dict_list, instruction, instruction_tag, answer_tag, answer_pos, answer_neg):
    # Transform each entry into the desired format for "inputs" and "target"
    transformed_data = []
    input_text = ""
    for item in dict_list:
        combined = item['positives'] + item['negatives']
        random.shuffle(combined)  # Randomly shuffle the positive and negative examples

        for ex in combined:
          input_text += f"{instruction_tag}: {instruction}\n"
          input_text += f"{ex}\n"
          answer = f'{answer_tag}: {answer_pos}' if ex in item['positives'] else f'{answer_tag}: {answer_neg}'
          input_text += f'{answer}\n'

        # The target answer remains the same
        input_text += f"{instruction_tag}: {instruction}\n"
        input_text += f"{item['target_question']}\n"
        input_text += f'{answer_tag}: \n'
        target = answer_pos if item['target_answer'] == "positives" else answer_neg

        transformed_data.append({'inputs': input_text, 'target': target})
        input_text = ""

    # Create a DataFrame
    df = pd.DataFrame(transformed_data)
    return df

In [39]:
df_with_instruct_label = build_dataframe_with_instruct_label(
    dict_list=processed_dataset,
    instruction="Label the following tweet based on whether it contains hate speech.",
    instruction_tag="Question",
    answer_tag="Answer",
    answer_pos="Hate",
    answer_neg="No Hate"
)

In [40]:
# Save the DataFrame as a pickle file
pickle_path = '/content/drive/MyDrive/Phi-2/data/eval/hate/hate_with_instruct_label.pkl'
df_with_instruct_label.to_pickle(pickle_path)

In [45]:
# Assuming df_with_instruct_label is your pandas DataFrame
dataset = Dataset.from_pandas(df_with_instruct_label)

In [46]:
repo_name = "hate-with-instruction-with-label"  # Choose a unique name
create_repo(repo_name, repo_type="dataset", private=False)  # Set `private=True` for a private dataset

# Push the train dataset
dataset.push_to_hub("AlanYky/hate-with-instruction-with-label")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/AlanYky/hate-with-instruction-with-label/commit/d5843636347c30c2681b96b1780b0b3fdb637675', commit_message='Upload dataset', commit_description='', oid='d5843636347c30c2681b96b1780b0b3fdb637675', pr_url=None, pr_revision=None, pr_num=None)

### Build the With Instruction and No Relavent Label

In [47]:
def build_dataframe_with_instruct_no_label(dict_list, instruction, instruction_tag, answer_tag):
    # Transform each entry into the desired format for "inputs" and "target"
    transformed_data = []
    input_text = ""
    for item in dict_list:
        answer_pos = generate_unique_symbol()
        answer_neg = generate_unique_symbol()

        combined = item['positives'] + item['negatives']
        random.shuffle(combined)  # Randomly shuffle the positive and negative examples

        for ex in combined:
          input_text += f"{instruction_tag}: {instruction}\n"
          input_text += f"{ex}\n"
          answer = f'{answer_tag}: {answer_pos}' if ex in item['positives'] else f'{answer_tag}: {answer_neg}'
          input_text += f'{answer}\n'

        # The target answer remains the same
        input_text += f"{instruction_tag}: {instruction}\n"
        input_text += f"{item['target_question']}\n"
        input_text += f'{answer_tag}: \n'
        target = answer_pos if item['target_answer'] == "positives" else answer_neg

        transformed_data.append({'inputs': input_text, 'target': target})
        input_text = ""

    # Create a DataFrame
    df = pd.DataFrame(transformed_data)
    return df

In [48]:
df_with_instruct_no_label = build_dataframe_with_instruct_no_label(
    dict_list=processed_dataset,
    instruction="Label the following tweet based on whether it contains hate speech.",
    instruction_tag="Question",
    answer_tag="Answer",
)


In [49]:
# Save the DataFrame as a pickle file
pickle_path = '/content/drive/MyDrive/Phi-2/data/eval/hate/hate_with_instruct_symbol.pkl'
df_with_instruct_no_label.to_pickle(pickle_path)

In [50]:
# Assuming df_with_instruct_label is your pandas DataFrame
dataset = Dataset.from_pandas(df_with_instruct_no_label)

In [51]:
repo_name = "hate-with-instruction-with-symbol"  # Choose a unique name
create_repo(repo_name, repo_type="dataset", private=False)  # Set `private=True` for a private dataset

# Push the train dataset
dataset.push_to_hub("AlanYky/hate-with-instruction-with-symbol")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/AlanYky/hate-with-instruction-with-symbol/commit/f979da3629dac3941a7af8560baa1875cfe26668', commit_message='Upload dataset', commit_description='', oid='f979da3629dac3941a7af8560baa1875cfe26668', pr_url=None, pr_revision=None, pr_num=None)

### Build the No Instruction and No Relavent Label

In [9]:
def build_dataframe_no_instruct_no_label(dict_list, instruction_tag, answer_tag):
    # Transform each entry into the desired format for "inputs" and "target"
    transformed_data = []
    input_text = ""
    for item in dict_list:
        answer_pos = generate_unique_symbol()
        answer_neg = generate_unique_symbol()

        combined = item['positives'] + item['negatives']
        random.shuffle(combined)  # Randomly shuffle the positive and negative examples

        for ex in combined:
          input_text += f"{instruction_tag}: {ex}\n"
          answer = f'{answer_tag}: {answer_pos}' if ex in item['positives'] else f'{answer_tag}: {answer_neg}'
          input_text += f'{answer}\n'

        # The target answer remains the same
        input_text += f"{instruction_tag}: {item['target_question']}\n"
        input_text += f'{answer_tag}: \n'
        target = answer_pos if item['target_answer'] == "positives" else answer_neg

        transformed_data.append({'inputs': input_text, 'target': target})
        input_text = ""

    # Create a DataFrame
    df = pd.DataFrame(transformed_data)
    return df

In [10]:
import json

# Define the file path
file_path = '/content/drive/MyDrive/Phi-2/data/eval/hate/hate_dataset.json'

# Read the data from the JSON file
with open(file_path, 'r') as f:
    processed_dataset = json.load(f)

In [11]:
df_with_instruct_no_instruct_no_label = build_dataframe_no_instruct_no_label(
    dict_list=processed_dataset,
    instruction_tag="Question",
    answer_tag="Answer",
)

In [12]:
# Save the DataFrame as a pickle file
pickle_path = '/content/drive/MyDrive/Phi-2/data/eval/hate/hate_no_instruct_symbol.pkl'
df_with_instruct_no_instruct_no_label.to_pickle(pickle_path)
# df_with_instruct_no_instruct_no_label = pd.read_pickle(pickle_path)

In [13]:
# Assuming df_with_instruct_label is your pandas DataFrame
dataset = Dataset.from_pandas(df_with_instruct_no_instruct_no_label)

In [14]:
repo_name = "hate-no-instruction-with-symbol"  # Choose a unique name
create_repo(repo_name, repo_type="dataset", private=False)  # Set `private=True` for a private dataset

# Push the train dataset
dataset.push_to_hub("AlanYky/hate-no-instruction-with-symbol")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/AlanYky/hate-no-instruction-with-symbol/commit/ed14b0b324b98207445249cb22fab932225d2fc1', commit_message='Upload dataset', commit_description='', oid='ed14b0b324b98207445249cb22fab932225d2fc1', pr_url=None, pr_revision=None, pr_num=None)

### Flipped Data

In [15]:
def build_dataframe_flipped(dict_list, instruction_tag, answer_tag, answer_pos, answer_neg):
    # Transform each entry into the desired format for "inputs" and "target"
    transformed_data = []
    input_text = ""
    for item in dict_list:
        combined = item['positives'] + item['negatives']
        random.shuffle(combined)  # Randomly shuffle the positive and negative examples

        for ex in combined:
          input_text += f"{instruction_tag}: {ex}\n"
          answer = f'{answer_tag}: {answer_neg}' if ex in item['positives'] else f'{answer_tag}: {answer_pos}' # Flipped Here
          input_text += f'{answer}\n'

        # The target answer remains the same
        input_text += f"{instruction_tag}: {item['target_question']}\n"
        input_text += f'{answer_tag}: \n'
        target = answer_neg if item['target_answer'] == "positives" else answer_pos

        transformed_data.append({'inputs': input_text, 'target': target})
        input_text = ""

    # Create a DataFrame
    df = pd.DataFrame(transformed_data)
    return df

In [16]:
df_with_flipped = build_dataframe_flipped(
    dict_list=processed_dataset,
    instruction_tag="Input",
    answer_tag="Symbol",
    answer_pos="Hate",
    answer_neg="Not Hate"
)

In [17]:
# Save the DataFrame as a pickle file
pickle_path = '/content/drive/MyDrive/Phi-2/data/eval/hate/hate_flipped.pkl'
df_with_flipped.to_pickle(pickle_path)

In [18]:
# Assuming df_with_instruct_label is your pandas DataFrame
dataset = Dataset.from_pandas(df_with_flipped)

In [19]:
repo_name = "hate-flipped"  # Choose a unique name
create_repo(repo_name, repo_type="dataset", private=False)  # Set `private=True` for a private dataset

# Push the train dataset
dataset.push_to_hub("AlanYky/hate-flipped")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/AlanYky/hate-flipped/commit/8eca6b3aa3f4662d9a6e0b475fee4df4204516ad', commit_message='Upload dataset', commit_description='', oid='8eca6b3aa3f4662d9a6e0b475fee4df4204516ad', pr_url=None, pr_revision=None, pr_num=None)

## CD (Climate)

- With Instruction, No Relevant Label
- No Instruction, No Relevant Label
- Flipped Data

In [81]:
climate_data = load_dataset("climatebert/climate_detection")

Generating train split:   0%|          | 0/1300 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400 [00:00<?, ? examples/s]

In [82]:
climate_data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1300
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 400
    })
})

In [84]:
climate = [item for item in climate_data['train'] if item['label'] == 1]
no_climate = [item for item in climate_data['train'] if item['label'] == 0]
print(f"climate: {len(climate)}")
print(f"no_climate: {len(no_climate)}")

climate: 1000
no_climate: 300


In [90]:
# Select the first 500 examples from the training set
subset = climate_data['train'].select(range(800))

climate = [item for item in subset if item['label'] == 1]
no_climate = [item for item in subset if item['label'] == 0]
print(f"climate: {len(climate)}")
print(f"no_climate: {len(no_climate)}")

climate: 648
no_climate: 152


### Build the subset

In [91]:
def build_examples_dataset(data, num_positives=4, num_negatives=4):
    examples_dataset = []
    count = 0
    for idx, target in enumerate(data):
        # Filter out the target text from the lists
        positive_examples = [item for item in data if item['label'] == 1 and item['text'] != target['text']] # climate-related
        negative_examples = [item for item in data if item['label'] == 0 and item['text'] != target['text']] # not climate-related

        # random select positive and negative
        selected_positives = random.sample(positive_examples, num_positives)
        selected_negatives = random.sample(negative_examples, num_negatives)

        # Assuming 'text' is your question and 'label' gives the answer (hate/no hate)
        example = {
            'positives': [ex['text'] for ex in selected_positives],
            'negatives': [ex['text'] for ex in selected_negatives],
            'target_question': target['text'],
            'target_answer': "positives" if target['label'] == 1 else "negatives"
        }

        examples_dataset.append(example)
        print(count)
        count += 1

    return examples_dataset

In [92]:
processed_dataset = build_examples_dataset(subset)
processed_dataset[0]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

{'positives': ['The physical risks of climate change are divided into acute and chronic risks. Acute risks include risks related to extreme weather events, such as floods and hurricanes. Chronic risks include, for example, permanently higher temperatures and the ensuing sea level rise. Sectors particularly exposed to physical risks include the forest sector, agriculture and real estate, to name a few.',
  'We are developing this approach through a pilot study to better understand the impact of different climate change pathways on our mortgage securities, housing association exposures and branch network to enable us to estimate the financial impact this may have.',
  'EXAMPLES OF RISKS Resource scarcity, coupled with increasing demand, could affect production, availability, quality and cost of raw materials. Increased frequency of extreme weather events, from floods to droughts, could cause disruption in our supply chain and impact the sourcing of raw materials, as well as the productio

In [93]:
# Save the dataset to a JSON file
file_path = '/content/drive/MyDrive/Phi-2/data/eval/climate/climate_dataset.json'
with open(file_path, 'w') as f:
    json.dump(processed_dataset, f)

### Build the With Instruction and With Relavent Label

In [94]:
def build_dataframe_with_instruct_label(dict_list, instruction, instruction_tag, answer_tag, answer_pos, answer_neg):
    # Transform each entry into the desired format for "inputs" and "target"
    transformed_data = []
    input_text = ""
    for item in dict_list:
        combined = item['positives'] + item['negatives']
        random.shuffle(combined)  # Randomly shuffle the positive and negative examples

        for ex in combined:
          input_text += f"{instruction_tag}: {instruction}\n"
          input_text += f"{ex}\n"
          answer = f'{answer_tag}: {answer_pos}' if ex in item['positives'] else f'{answer_tag}: {answer_neg}'
          input_text += f'{answer}\n'

        # The target answer remains the same
        input_text += f"{instruction_tag}: {instruction}\n"
        input_text += f"{item['target_question']}\n"
        input_text += f'{answer_tag}: \n'
        target = answer_pos if item['target_answer'] == "positives" else answer_neg

        transformed_data.append({'inputs': input_text, 'target': target})
        input_text = ""

    # Create a DataFrame
    df = pd.DataFrame(transformed_data)
    return df

In [95]:
df_with_instruct_label = build_dataframe_with_instruct_label(
    dict_list=processed_dataset,
    instruction="Label the following sentence based on whether it is related to climate.",
    instruction_tag="Question",
    answer_tag="Answer",
    answer_pos="Climate Event",
    answer_neg="Not Climate Event"
)

In [96]:
# Save the DataFrame as a pickle file
pickle_path = '/content/drive/MyDrive/Phi-2/data/eval/climate/climate_with_instruct_label.pkl'
df_with_instruct_label.to_pickle(pickle_path)

In [97]:
# Assuming df_with_instruct_label is your pandas DataFrame
dataset = Dataset.from_pandas(df_with_instruct_label)

In [98]:
repo_name = "climate-with-instruction-with-label"  # Choose a unique name
create_repo(repo_name, repo_type="dataset", private=False)  # Set `private=True` for a private dataset

# Push the train dataset
dataset.push_to_hub("AlanYky/climate-with-instruction-with-label")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/AlanYky/climate-with-instruction-with-label/commit/cb7cd04d8b1d95bc85cf8b8fabc852f5978b00a5', commit_message='Upload dataset', commit_description='', oid='cb7cd04d8b1d95bc85cf8b8fabc852f5978b00a5', pr_url=None, pr_revision=None, pr_num=None)

### Build the With Instruction and No Relavent Label

In [99]:
def build_dataframe_with_instruct_no_label(dict_list, instruction, instruction_tag, answer_tag):
    # Transform each entry into the desired format for "inputs" and "target"
    transformed_data = []
    input_text = ""
    for item in dict_list:
        answer_pos = generate_unique_symbol()
        answer_neg = generate_unique_symbol()

        combined = item['positives'] + item['negatives']
        random.shuffle(combined)  # Randomly shuffle the positive and negative examples

        for ex in combined:
          input_text += f"{instruction_tag}: {instruction}\n"
          input_text += f"{ex}\n"
          answer = f'{answer_tag}: {answer_pos}' if ex in item['positives'] else f'{answer_tag}: {answer_neg}'
          input_text += f'{answer}\n'

        # The target answer remains the same
        input_text += f"{instruction_tag}: {instruction}\n"
        input_text += f"{item['target_question']}\n"
        input_text += f'{answer_tag}: \n'
        target = answer_pos if item['target_answer'] == "positives" else answer_neg

        transformed_data.append({'inputs': input_text, 'target': target})
        input_text = ""

    # Create a DataFrame
    df = pd.DataFrame(transformed_data)
    return df

In [100]:
df_with_instruct_no_label = build_dataframe_with_instruct_no_label(
    dict_list=processed_dataset,
    instruction="Label the following sentence based on whether it is related to climate.",
    instruction_tag="Question",
    answer_tag="Answer",
)


In [101]:
# Save the DataFrame as a pickle file
pickle_path = '/content/drive/MyDrive/Phi-2/data/eval/climate/climate_with_instruct_symbol.pkl'
df_with_instruct_no_label.to_pickle(pickle_path)

In [102]:
# Assuming df_with_instruct_label is your pandas DataFrame
dataset = Dataset.from_pandas(df_with_instruct_no_label)

In [103]:
repo_name = "climate-with-instruction-with-symbol"  # Choose a unique name
create_repo(repo_name, repo_type="dataset", private=False)  # Set `private=True` for a private dataset

# Push the train dataset
dataset.push_to_hub("AlanYky/climate-with-instruction-with-symbol")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/AlanYky/climate-with-instruction-with-symbol/commit/a1729b6e1f37d9b1a0708d99c7514be67cc0dd07', commit_message='Upload dataset', commit_description='', oid='a1729b6e1f37d9b1a0708d99c7514be67cc0dd07', pr_url=None, pr_revision=None, pr_num=None)

### Build the No Instruction and No Relavent Label

In [104]:
def build_dataframe_no_instruct_no_label(dict_list, instruction_tag, answer_tag):
    # Transform each entry into the desired format for "inputs" and "target"
    transformed_data = []
    input_text = ""
    for item in dict_list:
        answer_pos = generate_unique_symbol()
        answer_neg = generate_unique_symbol()

        combined = item['positives'] + item['negatives']
        random.shuffle(combined)  # Randomly shuffle the positive and negative examples

        for ex in combined:
          input_text += f"{instruction_tag}: {ex}\n"
          answer = f'{answer_tag}: {answer_pos}' if ex in item['positives'] else f'{answer_tag}: {answer_neg}'
          input_text += f'{answer}\n'

        # The target answer remains the same
        input_text += f"{instruction_tag}: {item['target_question']}\n"
        input_text += f'{answer_tag}: \n'
        target = answer_pos if item['target_answer'] == "positives" else answer_neg

        transformed_data.append({'inputs': input_text, 'target': target})
        input_text = ""

    # Create a DataFrame
    df = pd.DataFrame(transformed_data)
    return df

In [105]:
df_with_instruct_no_instruct_no_label = build_dataframe_no_instruct_no_label(
    dict_list=processed_dataset,
    instruction_tag="Question",
    answer_tag="Answer",
)

In [106]:
# Save the DataFrame as a pickle file
pickle_path = '/content/drive/MyDrive/Phi-2/data/eval/climate/climate_no_instruct_symbol.pkl'
df_with_instruct_no_instruct_no_label.to_pickle(pickle_path)


In [107]:
# Assuming df_with_instruct_label is your pandas DataFrame
dataset = Dataset.from_pandas(df_with_instruct_no_instruct_no_label)

In [108]:
repo_name = "climate-no-instruction-with-symbol"  # Choose a unique name
create_repo(repo_name, repo_type="dataset", private=False)  # Set `private=True` for a private dataset

# Push the train dataset
dataset.push_to_hub("AlanYky/climate-no-instruction-with-symbol")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/AlanYky/climate-no-instruction-with-symbol/commit/f67264f1b78210689c498411e3c06cfc08c6d17b', commit_message='Upload dataset', commit_description='', oid='f67264f1b78210689c498411e3c06cfc08c6d17b', pr_url=None, pr_revision=None, pr_num=None)

### Flipped Data

In [109]:
def build_dataframe_flipped(dict_list, instruction_tag, answer_tag, answer_pos, answer_neg):
    # Transform each entry into the desired format for "inputs" and "target"
    transformed_data = []
    input_text = ""
    for item in dict_list:
        combined = item['positives'] + item['negatives']
        random.shuffle(combined)  # Randomly shuffle the positive and negative examples

        for ex in combined:
          input_text += f"{instruction_tag}: {ex}\n"
          answer = f'{answer_tag}: {answer_neg}' if ex in item['positives'] else f'{answer_tag}: {answer_pos}' # Flipped Here
          input_text += f'{answer}\n'

        # The target answer remains the same
        input_text += f"{instruction_tag}: {item['target_question']}\n"
        input_text += f'{answer_tag}: \n'
        target = answer_neg if item['target_answer'] == "positives" else answer_pos

        transformed_data.append({'inputs': input_text, 'target': target})
        input_text = ""

    # Create a DataFrame
    df = pd.DataFrame(transformed_data)
    return df

In [110]:
df_with_flipped = build_dataframe_flipped(
    dict_list=processed_dataset,
    instruction_tag="Input",
    answer_tag="Symbol",
    answer_pos="Climate Event",
    answer_neg="Not Climate Event"
)

In [111]:
# Save the DataFrame as a pickle file
pickle_path = '/content/drive/MyDrive/Phi-2/data/eval/climate/climate_flipped.pkl'
df_with_flipped.to_pickle(pickle_path)

In [112]:
# Assuming df_with_instruct_label is your pandas DataFrame
dataset = Dataset.from_pandas(df_with_flipped)

In [113]:
repo_name = "climate-flipped"  # Choose a unique name
create_repo(repo_name, repo_type="dataset", private=False)  # Set `private=True` for a private dataset

# Push the train dataset
dataset.push_to_hub("AlanYky/climate-flipped")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/AlanYky/climate-flipped/commit/435319be8e9279c228d613adb0b11f8162f2330f', commit_message='Upload dataset', commit_description='', oid='435319be8e9279c228d613adb0b11f8162f2330f', pr_url=None, pr_revision=None, pr_num=None)

## TC

- With Instruction, No Relevant Label
- No Instruction, No Relevant Label
- Flipped Data

In [7]:
offensive_data = load_dataset("tweet_eval", "offensive")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/11916 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/860 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1324 [00:00<?, ? examples/s]

In [8]:
offensive_data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 11916
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 860
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1324
    })
})

In [9]:
offensive = [item for item in offensive_data['train'] if item['label'] == 1]
no_offensive = [item for item in offensive_data['train'] if item['label'] == 0]
print(f"offensive: {len(offensive)}")
print(f"no_offensive: {len(no_offensive)}")

offensive: 3941
no_offensive: 7975


In [10]:
# Select the first 500 examples from the training set
subset = offensive_data['train'].select(range(2000))

# Shuffle the data
subset = subset.shuffle(seed=42)

offensive = [item for item in subset if item['label'] == 1]
no_offensive = [item for item in subset if item['label'] == 0]
print(f"offensive: {len(offensive)}")
print(f"no_offensive: {len(no_offensive)}")

offensive: 691
no_offensive: 1309


### Build the subset

In [11]:
def build_examples_dataset(data, num_positives=4, num_negatives=4):
    examples_dataset = []
    count = 0
    for idx, target in enumerate(data):
        # Filter out the target text from the lists
        positive_examples = [item for item in data if item['label'] == 1 and item['text'] != target['text']] # offensive
        negative_examples = [item for item in data if item['label'] == 0 and item['text'] != target['text']] # non offensive

        # random select positive and negative
        selected_positives = random.sample(positive_examples, num_positives)
        selected_negatives = random.sample(negative_examples, num_negatives)

        # Assuming 'text' is your question and 'label' gives the answer (hate/no hate)
        example = {
            'positives': [ex['text'] for ex in selected_positives],
            'negatives': [ex['text'] for ex in selected_negatives],
            'target_question': target['text'],
            'target_answer': "positives" if target['label'] == 1 else "negatives"
        }

        examples_dataset.append(example)
        print(count)
        count += 1

    return examples_dataset

In [12]:
processed_dataset = build_examples_dataset(subset)
processed_dataset[0]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

{'positives': ['@user She is such a witch.  All she needs is a broom',
  '@user Its interesting to see the hypocrisy with these liberals when last year it was discovered that there was a “Hush Fund” for Congressional  Sexual Scandals while working for the government',
  '.@USER is staffed with unhinged Liberals &amp; @user  This is a disgusting statement.',
  '@user He is so brutal. Hopefully he sells the team...'],
 'negatives': ['He is awesome!!!! So personable and I know these people so appreciate him being there. MAGA',
  '@user @user What new gun control laws would you like to see passed and what would they accomplish?',
  "@user @user @user @user @user @user Anonymous has always been a resistance tactic/idea. It doesn't matter who is leading they are an enemy since Anon doesn't take political sides. There are Anons who support the idea of Antifa and there are those who don't. But no one speaks for the entirety of Anonymous.",
  '@user She is a war criminal. By the Nuremberg stand

In [13]:
# Save the dataset to a JSON file
file_path = '/content/drive/MyDrive/Phi-2/data/eval/offensive/offensive_dataset.json'
with open(file_path, 'w') as f:
    json.dump(processed_dataset, f)

### Build the With Instruction and With Relavent Label

In [14]:
def build_dataframe_with_instruct_label(dict_list, instruction, instruction_tag, answer_tag, answer_pos, answer_neg):
    # Transform each entry into the desired format for "inputs" and "target"
    transformed_data = []
    input_text = ""
    for item in dict_list:
        combined = item['positives'] + item['negatives']
        random.shuffle(combined)  # Randomly shuffle the positive and negative examples

        for ex in combined:
          input_text += f"{instruction_tag}: {instruction}\n"
          input_text += f"{ex}\n"
          answer = f'{answer_tag}: {answer_pos}' if ex in item['positives'] else f'{answer_tag}: {answer_neg}'
          input_text += f'{answer}\n'

        # The target answer remains the same
        input_text += f"{instruction_tag}: {instruction}\n"
        input_text += f"{item['target_question']}\n"
        input_text += f'{answer_tag}: \n'
        target = answer_pos if item['target_answer'] == "positives" else answer_neg

        transformed_data.append({'inputs': input_text, 'target': target})
        input_text = ""

    # Create a DataFrame
    df = pd.DataFrame(transformed_data)
    return df

In [15]:
df_with_instruct_label = build_dataframe_with_instruct_label(
    dict_list=processed_dataset,
    instruction="Label the following tweet based on whether it contains a offensive content.",
    instruction_tag="Question",
    answer_tag="Answer",
    answer_pos="Offensive",
    answer_neg="Not Offensive"
)

In [16]:
# Save the DataFrame as a pickle file
pickle_path = '/content/drive/MyDrive/Phi-2/data/eval/offensive/offensive_with_instruct_label.pkl'
df_with_instruct_label.to_pickle(pickle_path)

In [17]:
# Assuming df_with_instruct_label is your pandas DataFrame
dataset = Dataset.from_pandas(df_with_instruct_label)

In [18]:
repo_name = "offensive-with-instruction-with-label"  # Choose a unique name
create_repo(repo_name, repo_type="dataset", private=False)  # Set `private=True` for a private dataset

# Push the train dataset
dataset.push_to_hub("AlanYky/offensive-with-instruction-with-label")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/AlanYky/offensive-with-instruction-with-label/commit/c107f04f71f6a0a37227ea431177cbf95e18cc23', commit_message='Upload dataset', commit_description='', oid='c107f04f71f6a0a37227ea431177cbf95e18cc23', pr_url=None, pr_revision=None, pr_num=None)

### Build the With Instruction and No Relavent Label

In [19]:
def build_dataframe_with_instruct_no_label(dict_list, instruction, instruction_tag, answer_tag):
    # Transform each entry into the desired format for "inputs" and "target"
    transformed_data = []
    input_text = ""
    for item in dict_list:
        answer_pos = generate_unique_symbol()
        answer_neg = generate_unique_symbol()

        combined = item['positives'] + item['negatives']
        random.shuffle(combined)  # Randomly shuffle the positive and negative examples

        for ex in combined:
          input_text += f"{instruction_tag}: {instruction}\n"
          input_text += f"{ex}\n"
          answer = f'{answer_tag}: {answer_pos}' if ex in item['positives'] else f'{answer_tag}: {answer_neg}'
          input_text += f'{answer}\n'

        # The target answer remains the same
        input_text += f"{instruction_tag}: {instruction}\n"
        input_text += f"{item['target_question']}\n"
        input_text += f'{answer_tag}: \n'
        target = answer_pos if item['target_answer'] == "positives" else answer_neg

        transformed_data.append({'inputs': input_text, 'target': target})
        input_text = ""

    # Create a DataFrame
    df = pd.DataFrame(transformed_data)
    return df

In [20]:
df_with_instruct_no_label = build_dataframe_with_instruct_no_label(
    dict_list=processed_dataset,
    instruction="Label the following tweet based on whether it contains a offensive content.",
    instruction_tag="Question",
    answer_tag="Answer",
)


In [21]:
# Save the DataFrame as a pickle file
pickle_path = '/content/drive/MyDrive/Phi-2/data/eval/offensive/offensive_with_instruct_symbol.pkl'
df_with_instruct_no_label.to_pickle(pickle_path)

In [22]:
# Assuming df_with_instruct_label is your pandas DataFrame
dataset = Dataset.from_pandas(df_with_instruct_no_label)

In [23]:
repo_name = "offensive-with-instruction-with-symbol"  # Choose a unique name
create_repo(repo_name, repo_type="dataset", private=False)  # Set `private=True` for a private dataset

# Push the train dataset
dataset.push_to_hub("AlanYky/offensive-with-instruction-with-symbol")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/AlanYky/offensive-with-instruction-with-symbol/commit/bcba885b256b08757684bedb13641c957328c483', commit_message='Upload dataset', commit_description='', oid='bcba885b256b08757684bedb13641c957328c483', pr_url=None, pr_revision=None, pr_num=None)

### Build the No Instruction and No Relavent Label

In [24]:
def build_dataframe_no_instruct_no_label(dict_list, instruction_tag, answer_tag):
    # Transform each entry into the desired format for "inputs" and "target"
    transformed_data = []
    input_text = ""
    for item in dict_list:
        answer_pos = generate_unique_symbol()
        answer_neg = generate_unique_symbol()

        combined = item['positives'] + item['negatives']
        random.shuffle(combined)  # Randomly shuffle the positive and negative examples

        for ex in combined:
          input_text += f"{instruction_tag}: {ex}\n"
          answer = f'{answer_tag}: {answer_pos}' if ex in item['positives'] else f'{answer_tag}: {answer_neg}'
          input_text += f'{answer}\n'

        # The target answer remains the same
        input_text += f"{instruction_tag}: {item['target_question']}\n"
        input_text += f'{answer_tag}: \n'
        target = answer_pos if item['target_answer'] == "positives" else answer_neg

        transformed_data.append({'inputs': input_text, 'target': target})
        input_text = ""

    # Create a DataFrame
    df = pd.DataFrame(transformed_data)
    return df

In [25]:
df_with_instruct_no_instruct_no_label = build_dataframe_no_instruct_no_label(
    dict_list=processed_dataset,
    instruction_tag="Question",
    answer_tag="Answer",
)

In [26]:
# Save the DataFrame as a pickle file
pickle_path = '/content/drive/MyDrive/Phi-2/data/eval/offensive/offensive_no_instruct_symbol.pkl'
df_with_instruct_no_instruct_no_label.to_pickle(pickle_path)

In [27]:
# Assuming df_with_instruct_label is your pandas DataFrame
dataset = Dataset.from_pandas(df_with_instruct_no_instruct_no_label)

In [28]:
repo_name = "offensive-no-instruction-with-symbol"  # Choose a unique name
create_repo(repo_name, repo_type="dataset", private=False)  # Set `private=True` for a private dataset

# Push the train dataset
dataset.push_to_hub("AlanYky/offensive-no-instruction-with-symbol")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/AlanYky/offensive-no-instruction-with-symbol/commit/1453271031cfed9d0d4550a1c1b0f555a90898db', commit_message='Upload dataset', commit_description='', oid='1453271031cfed9d0d4550a1c1b0f555a90898db', pr_url=None, pr_revision=None, pr_num=None)

### Flipped Data

In [29]:
def build_dataframe_flipped(dict_list, instruction_tag, answer_tag, answer_pos, answer_neg):
    # Transform each entry into the desired format for "inputs" and "target"
    transformed_data = []
    input_text = ""
    for item in dict_list:
        combined = item['positives'] + item['negatives']
        random.shuffle(combined)  # Randomly shuffle the positive and negative examples

        for ex in combined:
          input_text += f"{instruction_tag}: {ex}\n"
          answer = f'{answer_tag}: {answer_neg}' if ex in item['positives'] else f'{answer_tag}: {answer_pos}' # Flipped Here
          input_text += f'{answer}\n'

        # The target answer remains the same
        input_text += f"{instruction_tag}: {item['target_question']}\n"
        input_text += f'{answer_tag}: \n'
        target = answer_neg if item['target_answer'] == "positives" else answer_pos

        transformed_data.append({'inputs': input_text, 'target': target})
        input_text = ""

    # Create a DataFrame
    df = pd.DataFrame(transformed_data)
    return df

In [30]:
df_with_flipped = build_dataframe_flipped(
    dict_list=processed_dataset,
    instruction_tag="Input",
    answer_tag="Symbol",
    answer_pos="Offensive",
    answer_neg="Not Offensive"
)

In [31]:
# Save the DataFrame as a pickle file
pickle_path = '/content/drive/MyDrive/Phi-2/data/eval/offensive/offensive_flipped.pkl'
df_with_flipped.to_pickle(pickle_path)

In [32]:
# Assuming df_with_instruct_label is your pandas DataFrame
dataset = Dataset.from_pandas(df_with_flipped)

In [33]:
repo_name = "offensive-flipped"  # Choose a unique name
create_repo(repo_name, repo_type="dataset", private=False)  # Set `private=True` for a private dataset

# Push the train dataset
dataset.push_to_hub("AlanYky/offensive-flipped")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/AlanYky/offensive-flipped/commit/d71ab26722106b6aa336050bdb6deedddbf02b74', commit_message='Upload dataset', commit_description='', oid='d71ab26722106b6aa336050bdb6deedddbf02b74', pr_url=None, pr_revision=None, pr_num=None)